# Train a TensorFlow 2.x model with the Amazon SageMaker optimized TensorFlow container and debug using Amazon SageMaker Debugger

[Amazon SageMaker](https://aws.amazon.com/sagemaker/) is a fully managed machine learning service. With SageMaker, you have the option of using the built-in algorithms as well as bringing your own algorithms and frameworks.  One such framework is TensorFlow 2.x.  [Amazon SageMaker Debugger](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html) debugs, monitors and profiles training jobs in real time thereby helping with detecting non-converging conditions, optimizing resource utilization by eliminating bottlenecks, improving training time and reducing costs of your machine learning models.

This notebook demonstrates how to use a SageMaker optimized TensorFlow 2.x container to train a multi-class image classification model using the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) and debug using SageMaker Debugger.  Finally the debugger's output is analyzed.  This will take your training script and use SageMaker in script mode.

This notebook will create resources in the same AWS account and in the same region where this notebook is running.

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Check and upgrade required software versions](#Check%20and%20upgrade%20required%20software%20versions)
    
    3. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)

    4. [Organize imports](#Organize%20imports)
    
    5. [Create common objects](#Create%20common%20objects)

2. [Prepare the dataset](#Prepare%20the%20dataset)

    1. [Create the local directories](#Create%20the%20local%20directories)

    2. [Load the dataset](#Load%20the%20dataset)
    
    3. [View the details of the dataset](#View%20the%20details%20of%20the%20dataset)
    
    4. [Visualize the dataset](#Visualize%20the%20dataset)
    
    5. [Normalize the dataset](#Normalize%20the%20dataset)
    
    6. [Save the prepared datasets locally](#Save%20the%20prepared%20datasets%20locally)
    
    7. [Upload the prepared datasets to S3](#Upload%20the%20prepared%20datasets%20to%20S3)

3. [View the training script](#View%20the%20training%20script)

4. [Perform training, validation and testing](#Perform%20training%20validation%20and%20testing)

    1. [Set the training parameters](#Set%20the%20training%20parameters)
    
    2. [Set the debugger parameters](#Set%20the%20debugger%20parameters)

    3. [Run the training job](#Run%20the%20training%20job)

5. [View the auto-generated debugger profiling report](#View%20the%20auto-generated%20debugger%20profiling%20report)

6. [Perform custom analysis of the debugger output](#Perform%20custom%20analysis%20of%20the%20debugger%20output)

    1. [Get the training job](#Get%20the%20training%20job)

    2. [Read the metrics](#Read%20the%20metrics)

    3. [Plot the metrics](#Plot%20the%20metrics)
    
        1. [System metrics histogram](#System%20metrics%20histogram)

        2. [Framework metrics stepline chart](#Framework%20metrics%20stepline%20chart)
        
        3. [Framework metrics step histogram](#Framework%20metrics%20step%20histogram)

        4. [System and framework metrics timeline charts](#System%20and%20framework%20metrics%20timeline%20charts)

        5. [System and framework metrics heatmap](#System%20and%20framework%20metrics%20heatmap)

7. [Cleanup](#Cleanup)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates and to download the dataset.  You can either provide direct Internet access (default) or provide Internet access through a VPC.  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

###  B. Check and upgrade required software versions <a id ='Check%20and%20upgrade%20required%20software%20versions'> </a>

This notebook requires:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [TensorFlow version 2.x with SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/tf.html)
* [Python 3.6.x](https://www.python.org/downloads/release/python-360/)
* [SMDebug](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-analyze-data.html)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)

Note: If you get 'module not found' errors in the following cell, then uncomment the appropriate installation commands and install the modules.  Also, uncomment and run the kernel shutdown command.  When the kernel comes back, comment out the installation and kernel shutdown commands and run the following cell.  Now, you should not see any errors.

In [ ]:
import boto3
import IPython
import sagemaker
import smdebug
import sys
import tensorflow as tf

# Install/upgrade sagemaker, boto3 smdebug and tensorflow
#!{sys.executable} -m pip install -U sagemaker boto3 smdebug
#!{sys.executable} -m pip install -U tensorflow
#IPython.Application.instance().kernel.do_shutdown(True)

# Get the current installed version of Sagemaker SDK, TensorFlow, Python and Boto3
print('SageMaker Python SDK version : {}'.format(sagemaker.__version__))
print('TensorFlow version : {}'.format(tf.__version__))
print('Python version : {}'.format(sys.version))
print('Boto3 version : {}'.format(boto3.__version__))

###  C. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell.

Note: This role should have the following permissions,

1. Full access to the S3 bucket that will be used to store training and output data.
2. Full access to launch training instances.
3. Access to write to CloudWatch.

In [ ]:
print(sagemaker.get_execution_role())

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [ ]:
from IPython.core.display import display, HTML
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from sagemaker.debugger import (ProfilerConfig,
                                FrameworkProfile,
                                CollectionConfig,
                                DebuggerHookConfig,
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig,
                                Rule,
                                PythonProfiler,
                                cProfileTimer,
                                ProfilerRule,
                                rule_configs)
from sagemaker.tensorflow import TensorFlow
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram
from smdebug.profiler.analysis.notebook_utils.step_timeline_chart import StepTimelineChart
from smdebug.profiler.analysis.notebook_utils.step_histogram import StepHistogram
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
import time

###  E. Create common objects <a id='Create%20common%20objects'></a>

Create common objects to be used in future steps in this notebook.

In [ ]:
# Specify the S3 bucket name
s3_bucket = '<Specify the S3 bucket name>'

# Create the S3 Boto3 resource
s3_resource = boto3.resource('s3')
s3_bucket_resource = s3_resource.Bucket(s3_bucket)

# Base name to be used to create resources
nb_name = 'tf2-fashion-mnist-debugger'

# Names of various resources
train_job_name = 'train-{}'.format(nb_name)

# Names of local sub-directories in the notebook file system
data_dir = os.path.join(os.getcwd(), 'data/{}'.format(nb_name))
train_dir = os.path.join(os.getcwd(), 'data/{}/train'.format(nb_name))
test_dir = os.path.join(os.getcwd(), 'data/{}/test'.format(nb_name))

# Sub-folder names in S3
train_dir_s3_prefix = '{}/data/train'.format(nb_name)
test_dir_s3_prefix = '{}/data/test'.format(nb_name)

# Location in S3 where the training scripts will be copied
code_location = 's3://{}/{}/scripts'.format(s3_bucket, nb_name)

# Location in S3 where the model checkpoint will be stored
model_checkpoint_s3_path = 's3://{}/{}/checkpoint/'.format(s3_bucket, nb_name)

# Location in S3 where the trained model and debugger output will be stored
model_and_debugger_output_s3_path = 's3://{}/{}/output/'.format(s3_bucket, nb_name)

## 2. Prepare the dataset <a id ='Prepare%20the%20dataset'> </a>

The [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) consists of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images.  These categories are mapped to integers from 0 to 9 and represent the following class labels,

* 0: T-shirt/top
* 1: Trouser
* 2: Pullover
* 3: Dress
* 4: Coat
* 5: Sandal
* 6: Shirt
* 7: Sneaker
* 8: Bag
* 9: Ankle boot

The following steps will help with preparing the dataset for training.

### A) Create the local directories <a id='Create%20the%20local%20directories'></a>

Create the directories in the local system where the dataset will be copied to and processed.

In [ ]:
# Create the local directories
os.makedirs(data_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

### B) Load the dataset <a id ='Load%20the%20dataset'> </a>

Load the pre-shuffled train and test data with the keras.datasets API.

In [ ]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

### C) View the details of the dataset <a id ='View%20the%20details%20of%20the%20dataset'> </a>

Print the shape of the data and you will notice that they are 28x28 pixels.  There are 60,000 images in the training data and 10,000 images in the test data.

In [ ]:
# Summarize the dataset
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

### D) Visualize the dataset <a id ='Visualize%20the%20dataset'> </a>

Randomly display the images and labels of `sample_size` number of images from the test dataset.

In [ ]:
# Randomly display the images and labels of n (sample_size) images from the test dataset

sample_size = 50

random_indexes = np.random.randint(0, len(x_test), sample_size)
sample_images = x_test[random_indexes]
sample_labels = y_test[random_indexes]
sample_predictions = None
num_rows = 5
num_cols = 10
plot_title = None
fig_size = None
assert sample_images.shape[0] == num_rows * num_cols

# Labels
FASHION_LABELS = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

import seaborn as sns

with sns.axes_style("whitegrid"):
    sns.set_context("notebook", font_scale=1.1)
    sns.set_style({"font.sans-serif": ["Verdana", "Arial", "Calibri", "DejaVu Sans"]})
    f, ax = plt.subplots(num_rows, num_cols, figsize=((14, 9) if fig_size is None else fig_size),
        gridspec_kw={"wspace": 0.02, "hspace": 0.30}, squeeze=True)
    for r in range(num_rows):
        for c in range(num_cols):
            image_index = r * num_cols + c
            ax[r, c].axis("off")
            ax[r, c].imshow(sample_images[image_index], cmap="Greys")
            if sample_predictions is None:
                title = ax[r, c].set_title("%s" % FASHION_LABELS[sample_labels[image_index]])
            else:
                true_label = sample_labels[image_index]
                pred_label = sample_predictions[image_index]
                prediction_matches_true = (sample_labels[image_index] == sample_predictions[image_index])
                if prediction_matches_true:
                    title = FASHION_LABELS[true_label]
                    title_color = 'g'
                else:
                    title = '%s/%s' % (FASHION_LABELS[true_label], FASHION_LABELS[pred_label])
                    title_color = 'r'
                title = ax[r, c].set_title(title)
                plt.setp(title, color=title_color)
    if plot_title is not None:
        f.suptitle(plot_title)
    plt.show()
    plt.close()

The pixel values of the images fall in the range of 0 to 255. You can verify this below.

In [ ]:
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

### E) Normalize the dataset <a id ='Normalize%20the%20dataset'> </a>

As the pixel values range from 0 to 255, it is important to normalize them to a range from 0 to 1.  This can be done by dividing these values by 255.  This has to be done for both training and test images.

In [ ]:
# Normalize the dataset
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

### F) Save the prepared datasets locally <a id='Save%20the%20prepared%20datasets%20locally'></a>

Save the prepared train, validate and test datasets to local directories.  Prior to saving, concatenate x and y columns as needed.  Create the directories if they don't exist.

In [ ]:
# Save the prepared dataset (in numpy format) to the local directories
np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)

### G) Upload the prepared datasets to S3 <a id ='Upload%20the%20prepared%20datasets%20to%20S3'> </a>

Upload the datasets from the local directories to appropriate sub-directories in the specified S3 bucket.

In [ ]:
# Upload the data to S3
train_data_s3_full_path = sagemaker.Session().upload_data(path='./data/{}/train/'.format(nb_name),
                                                          bucket=s3_bucket,
                                                          key_prefix=train_dir_s3_prefix)
test_data_s3_full_path = sagemaker.Session().upload_data(path='./data/{}/test/'.format(nb_name),
                                                         bucket=s3_bucket,
                                                         key_prefix=test_dir_s3_prefix)

## 3. View the training script <a id ='View%20the%20training%20script'> </a>

View the script that will be used for training the model.  This should exist in a local directory.

In [ ]:
!cat scripts/train_tf2_fashion_mnist.py

##  4. Perform training, validation and testing <a id ='Perform%20training%20validation%20and%20testing'> </a>

In this step, we will use a SageMaker optimized TensorFlow 2.x container to train a multi-class image classification model using the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).  This will take your training script and use SageMaker in script mode.

Debugger will be enabled as part of the training process.

### A) Set the training parameters <a id ='Set%20the%20training%20parameters'> </a>

1. Inputs - S3 locations for training and test data.
2. Hyperparameters.
3. Training instance details:

    1. Instance count
    
    2. Instance type
    
    3. The max run time of the training job
    
    4. (Optional) Use Spot instances.  For more info, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html).
    
    5. (Optional) The max wait for Spot instances, if using Spot.  This should be larger than the max run time.
    
4. Tensorflow framework version and Python version.
5. Names of the training script and the local directory where it is located.
6. Logging level of the SageMaker optimized Tensorflow 2.x container.
7. Appropriate local and S3 directories that will be used by the training job.

In [ ]:
# Set the input data paths
inputs = {'train':train_data_s3_full_path, 'test':test_data_s3_full_path}

# Set the hyperparameters
hyperparameters = {'epochs': 25,
                   'batch_size': 32,
                   'learning_rate': 0.001,
                   'decay': 1e-6}

# Set the instance count, instance type, options to use Spot instances, job name and other parameters
train_instance_count = 1
train_instance_type = 'ml.p3.16xlarge'
#use_spot_instances = True
#spot_max_wait_time_in_seconds = 5400
use_spot_instances = False
spot_max_wait_time_in_seconds = None
max_run_time_in_seconds = 3600

framework_version = '2.4.1'
py_version = 'py37'

# Set the training script related parameters
train_script_dir = 'scripts'
train_script = 'train_tf2_fashion_mnist.py'

# Set the training container related parameters
container_log_level = logging.INFO

# Location where the model checkpoints will be stored locally in the container before being uploaded to S3
model_checkpoint_local_dir = '/opt/ml/checkpoints/'

# Location where the trained model will be stored locally in the container before being uploaded to S3
model_local_dir = '/opt/ml/model'

### B) Set the debugger parameters <a id ='Set%20the%20debugger%20parameters'> </a>

1. **Profile config** - configure how to collect system metrics and framework metrics from your training job and save into your secured S3 bucket URI or local machine.

    1. [Monitoring hardware system resource utilization](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configure-system-monitoring.html)
  
    2. [Framework profiling](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configure-framework-profiling.html)
  
2. **Debugger hook config** - configure how to collect output tensors from your training job and save into your secured S3 bucket URI or local machine.  For more info, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-configure-hook.html).

3. **Rules** - configure this parameter to enable Debugger built-in rules that you want to run in parallel. The rules automatically analyze your training job and find training issues. The ProfilerReport rule saves the Debugger profiling reports in your secured S3 bucket URI.  For more info, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/use-debugger-built-in-rules.html).

In [ ]:
# Location in S3 where the debugger output will be stored is mentioned in the previous step

# Set the profile config for both system and framework metrics
profiler_config = ProfilerConfig(
    system_monitor_interval_millis = 1000,
    framework_profile_params = FrameworkProfile(
        detailed_profiling_config = DetailedProfilingConfig(
            start_step = 5, 
            num_steps = 10
        ),
        dataloader_profiling_config = DataloaderProfilingConfig(
            start_step = 7, 
            num_steps = 10
        ),
        python_profiling_config = PythonProfilingConfig(
            start_step = 9, 
            num_steps = 10,
            python_profiler = PythonProfiler.CPROFILE, 
            cprofile_timer = cProfileTimer.TOTAL_TIME
        )
    )
)

# Set the debugger hook config to save tensors
debugger_hook_config = DebuggerHookConfig(
    collection_configs = [
        CollectionConfig(name = "weights"),
        CollectionConfig(name = "gradients")
    ]
)

# Set the rules to analyze tensors emitted during training
rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.stalled_training_rule())
]


### C) Run the training job <a id='Run%20the%20training%20job'></a>

Prepare the `estimator` and call the `fit()` method.  This will pull the container containing the specified version of the framework in the AWS region, copy the training script to it and run the training job in the specified type of EC2 instance(s).  The training data will be pulled from the specified location in S3 and the generated model along with the checkpoints will be written to the specified locations in S3.  The debugger will use its configured settings to capture the data and write them to the specified locations in S3.

In [ ]:
# Create the estimator
estimator = TensorFlow(
    source_dir=train_script_dir,
    entry_point=train_script,
    code_location=code_location,
    checkpoint_local_path=model_checkpoint_local_dir,
    checkpoint_s3_uri=model_checkpoint_s3_path,
    model_dir=model_local_dir,
    output_path=model_and_debugger_output_s3_path,
    instance_type=train_instance_type,
    instance_count=train_instance_count,
    use_spot_instances=use_spot_instances,
    max_wait=spot_max_wait_time_in_seconds,
    max_run=max_run_time_in_seconds,
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    base_job_name=train_job_name,
    framework_version=framework_version,
    py_version=py_version,
    container_log_level=container_log_level,
    script_mode=True,
    disable_profiler=False,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_hook_config,
    rules=rules)

# Perform the training
estimator.fit(inputs, wait=True)

## 5. View the auto-generated debugger profiling report <a id ='View%20the%20auto-generated%20debugger%20profiling%20report'> </a>

The debugger's auto-generated profiling report will be stored in the S3 directory specified in earlier steps.  You can view it here.

For information on how to read the report, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-profiling-report.html).

In [ ]:
# Get the S3 path to the debugger's auto-generated profiling report
profiling_report_s3_prefix = '{}/output/{}/rule-output/ProfilerReport/profiler-output/profiler-report.html'.format(nb_name,
                                                                             estimator.latest_training_job.job_name)
profiling_report = sagemaker.Session().read_s3_file(s3_bucket, profiling_report_s3_prefix)


# Print debugger's auto-generated profiling report location
display(HTML(profiling_report))

## 6. Perform custom analysis of the debugger output <a id ='Perform%20custom%20analysis%20of%20the%20debugger%20output'> </a>

The debugger's output will be stored in the S3 directories specified in earlier steps.  In this step, we will read that data and display them through various visualizations.

### A. Get the training job <a id ='Get%20the%20training%20job'> </a>

Get the training job object from the estimator object used for training in the previous step.  This is required to read the metrics from the debugger's output.

In [ ]:
# This assumes that the job was trained in the same AWS region as the S3 bucket where the debugger output is stored
# If not, then make appropriate changes to the following code
tj = TrainingJob(estimator.latest_training_job.job_name, sagemaker.Session().boto_region_name)

### B. Read the metrics <a id ='Read%20the%20metrics'> </a>

1. Wait for the the system and framework metrics to be available.
2. Get the reader objects for both of these metrics.
3. Refresh the event file lists that contains these metrics.

In [ ]:
# Wait for the data to be available
tj.wait_for_sys_profiling_data_to_be_available()
tj.wait_for_framework_profiling_data_to_be_available()
# Get the metrics reader
system_metrics_reader = tj.get_systems_metrics_reader()
framework_metrics_reader = tj.get_framework_metrics_reader()
# Refresh the event file list
system_metrics_reader.refresh_event_file_list()
framework_metrics_reader.refresh_event_file_list()

### C. Plot the metrics <a id ='Plot%20the%20metrics'> </a>

Plot visualizations for the metrics read in the previous step.

#### a. System metrics histogram <a id ='System%20metrics%20histogram'> </a>

In [ ]:
metrics_histogram = MetricsHistogram(system_metrics_reader)
metrics_histogram.plot(
    starttime=0, 
    endtime=system_metrics_reader.get_timestamp_of_latest_available_file(), 
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"]
)

#### b. Framework metrics stepline chart <a id ='Framework%20metrics%20stepline%20chart'> </a>

In [ ]:
view_step_timeline_chart = StepTimelineChart(framework_metrics_reader)

#### c. Framework metrics step histogram <a id ='Framework%20metrics%20step%20histogram'> </a>

In [ ]:
step_histogram = StepHistogram(framework_metrics_reader)
step_histogram.plot(
    starttime=step_histogram.last_timestamp - 5 * 1000 * 1000, 
    endtime=step_histogram.last_timestamp, 
    show_workers=True
)

#### d. System and framework metrics timeline charts <a id ='System%20and%20framework%20metrics%20timeline%20charts'> </a>

In [ ]:
view_timeline_charts = TimelineCharts(
    system_metrics_reader, 
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"] 
)

view_timeline_charts.plot_detailed_profiler_data([700,710]) 

#### e. System and framework metrics heatmap <a id ='System%20and%20framework%20metrics%20heatmap'> </a>

In [ ]:
view_heatmap = Heatmap(
    system_metrics_reader,
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"],
    plot_height=450
)

## 7. Cleanup <a id='Cleanup'></a>

As a best practice, you should delete resources and S3 objects when no longer required.  This will help you avoid incurring unncessary costs.

This step will cleanup the resources and S3 objects created by this notebook.

In [ ]:
# Delete data from S3 bucket
for file in s3_bucket_resource.objects.filter(Prefix='{}/'.format(nb_name)):
    file_key = file.key
    print('Deleting {} ...'.format(file_key))
    s3_resource.Object(s3_bucket_resource.name, file_key).delete()